In [416]:
import torch
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [444]:
class value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"value(data = {self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, value) else value(other)
        out = value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        other = other if isinstance(other, value) else value(other)
        out = value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other-1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * (-1)
    
    def __sub__(self, other):
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1) / (math.exp(2*x)+1)
        out = value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1-t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        
        out._backward = _backward
                    
        return out            
            
        
    def backward(self):

        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


In [445]:
from graphviz import Digraph

def trace(root):
    # builds a set of all nodes and edges in a graph
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation, create an op node for it
            dot.node(name = uid + n._op, label = n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [446]:
x1 = torch.Tensor([2.0]).double()                 ; x1.requires_grad = True
x2 = torch.Tensor([0.0]).double()                 ; x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double()                ; w1.requires_grad = True
w2 = torch.Tensor([1.0]).double()                 ; w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double()   ; b.requires_grad = True

n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('x1', x1.grad.item())
print('w1', w1.grad.item())
print('x2', x2.grad.item())
print('w2', w2.grad.item())

0.7071066904050358
x1 -1.5000003851533106
w1 1.0000002567688737
x2 0.5000001283844369
w2 0.0


In [447]:
import random
class Neuron:
    def __init__(self, nin):
        self.w = [value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layers in self.layers for p in layers.parameters()]
     

In [448]:
# at Single Neuron 
x = [2.0, 3.0]     
n = Neuron(2)
n(x)

value(data = 0.8925046051771063)

In [449]:
# Layer neurons
x = [2.0, 3.0]     
n = Layer(2, 3)
n(x)

[value(data = 0.40232594024973267),
 value(data = 0.6860868199151923),
 value(data = 0.8034147959709924)]

In [450]:
# MLP neurons

x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

value(data = -0.7512251057876331)

In [494]:
n.parameters()

[value(data = -0.8766987128088563),
 value(data = -0.04247444456455958),
 value(data = 0.7150924118580467),
 value(data = -0.511775456516705),
 value(data = 0.3621079307736545),
 value(data = -0.15791962722043273),
 value(data = 1.1046267042817648),
 value(data = -0.2948825154196752),
 value(data = 0.4560884363393834),
 value(data = -0.05413440603578145),
 value(data = 1.1511965017867485),
 value(data = -0.3105362025313491),
 value(data = -0.7119811634809805),
 value(data = -0.09437708376890092),
 value(data = -1.0896014960381755),
 value(data = -0.3330287824274714),
 value(data = 0.1709421290348547),
 value(data = 1.1993721450150974),
 value(data = 1.093671367802274),
 value(data = -0.30562831352831465),
 value(data = -0.3718736048724482),
 value(data = -0.8689916734951494),
 value(data = 0.3050921184751729),
 value(data = -0.6885837040323883),
 value(data = -0.2403309372255974),
 value(data = 1.108217788084576),
 value(data = -0.9130807484214469),
 value(data = 0.08819988021127939),


In [495]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets


In [801]:
for k in range(20):
    
    # Forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
    
    # Backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    
    print(k, loss.data)

0 0.00013639161675819335
1 0.00013636222993482073
2 0.0001363328543722679
3 0.00013630349006478414
4 0.000136274137006624
5 0.00013624479519204374
6 0.00013621546461529746
7 0.00013618614527065293
8 0.00013615683715238102
9 0.00013612754025474127
10 0.0001360982545720151
11 0.0001360689800984775
12 0.0001360397168284089
13 0.00013601046475609604
14 0.00013598122387582223
15 0.00013595199418188386
16 0.00013592277566857252
17 0.00013589356833018952
18 0.00013586437216103642
19 0.00013583518715541565


In [802]:
ypred

[value(data = 0.9935360795849149),
 value(data = -0.9941377934452217),
 value(data = -0.9933475444932565),
 value(data = 0.9960716046453071)]